# Visualization

In [ ]:
import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets  # for interactive plots

os.environ["OMP_NUM_THREADS"] = "2"
import gvec

Load a specific GVEC equilibrium state:

In [ ]:
rundir = Path("run_w7x_01")
# or rundir = Path(tokamak_gvecrun/run_01")
state = gvec.find_state(rundir)

pyGVEC allows computing a variety of different quantities. We can print a table of the available quantities which we can then use later:

In [ ]:
gvec.table_of_quantities(markdown=True)

Now we load a GVEC equilibrium, specifiy the resolution at which we want to evaluate it and the quantities of interest.
pyGVEC will now compute hte quantities of interest and their requirements and add them all to one `xarray.Dataset`.

In [ ]:
varlist = ["N_FP", "mod_B", "iota", "p", "I_tor", "I_pol", "iota_curr", "iota_0", "Phi", "chi"]
ev = state.evaluate(*varlist, rho=21, theta=51, zeta=51)

ev

The data contained in the `xarrray.Dataset` can now be plotted or used as desired.

First we plot the relevant profiles.

In [ ]:
# === Profiles === #
fig, axs = plt.subplots(2, 3, figsize=(15, 6), tight_layout=True, sharex=True)

for ax, var in zip(axs.flatten(), ["Phi", "chi", "iota", "p", "I_tor", "I_pol"]):
    ax.plot(ev.rho, ev[var], label=f"${ev[var].attrs['symbol']}$")
    ax.set(
        title=ev[var].attrs["long_name"],
        ylabel=f"${ev[var].attrs['symbol']}$",
    )

axs[0, 2].plot(ev.rho, ev.iota_curr, "--", label=f"${ev.iota_curr.attrs['symbol']}$")
axs[0, 2].plot(ev.rho, ev.iota_0, "--", label=f"${ev.iota_0.attrs['symbol']}$")

for ax in axs[-1, :]:
    ax.set_xlabel(f"${ev.rho.attrs['symbol']}$")

for ax in axs.flat:
    ax.legend()

And the cross-sections of the poloidal planes:

In [ ]:
# === Cross-sections === #
fig, axs = plt.subplots(3, 4, figsize=(15, 6), tight_layout=True, sharex=True, sharey=True)

cuts = np.linspace(0, 2 * np.pi / ev.N_FP.item(), 12)
for ax, zeta in zip(axs.flat, cuts):
    ds = ev.sel(zeta=zeta, method="nearest")

    r = np.linspace(0, ds.rad.size - 1, 5, dtype=int)
    ax.plot(ds.isel(rad=r).X1.T, ds.isel(rad=r).X2.T, "C0-")

    t = np.linspace(0, ds.pol.size - 1, 8, dtype=int, endpoint=False)
    ax.plot(ds.isel(pol=t).X1, ds.isel(pol=t).X2, "C1-")
    ax.set(
        aspect="equal",
        title=f"$\\zeta = {zeta:.2f}$",
    )

for ax in axs[-1, :]:
    ax.set_xlabel(f"${ev.X1.attrs['symbol']}$")
for ax in axs[:, 0]:
    ax.set_ylabel(f"${ev.X2.attrs['symbol']}$")

Lastly we can plot various quantities on a specific fluxsurface interactively:

In [ ]:
def plot_ds(fig, ax, ds, var, theta, zeta, dots=True, mode="color"):
    if mode in ["color", "both"]:
        pcm = ax.pcolormesh(zeta, theta, ds[var], shading="gouraud")
        fig.colorbar(pcm, ax=ax)
    if mode == "contour":
        cnt = ax.contour(zeta, theta, ds[var], 20)
        fig.colorbar(cnt, ax=ax)
    elif mode == "both":
        ax.contour(zeta, theta, ds[var], 20, colors="k", linewidths=0.5)

    if dots:
        t, z = np.meshgrid(zeta, theta)
        ax.plot(t.flat, z.flat, "k.", ms=1)
    ax.set(
        xlabel=f"${zeta.attrs['symbol']}$",
        ylabel=f"${theta.attrs['symbol']}$",
    )


def plot(rho, var, dots, mode, ds):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8), tight_layout=True, sharex=True, sharey=True)
    fig.suptitle(
        f"Logical coordinates, one field period, $N_{{FP}}={ds['N_FP'].item()}$, $\\rho={rho:.3f}$\n{var}: ${ds[var].attrs['symbol']}$\n{ds[var].attrs['long_name']}"
    )
    plot_ds(fig, ax, ds.sel(rho=rho), var, ds.theta, ds.zeta, dots, mode)


interact(
    plot,
    rho=widgets.SelectionSlider(options=ev.rho.data, continuous_update=False),
    var=[var for var in ev.data_vars if ev[var].dims == ("rad", "pol", "tor")],
    dots=[False, True],
    mode=["both", "color", "contour"],
    ds=widgets.fixed(ev),
);

For 3D visualizations one might want to change to powerful 3D visualization tools such as [paraview](https://www.paraview.org/). For such purposes pyGVEC datasets can easily be exported to the `.vts` format via the `ev2vtk` function. Note that all variables will be broadcast to the 3D grid defined by the `pos` variable. Hence, it might be advisable to select just those variables which will be used during the visualization to avoid unnecessary large files.

In [ ]:
from gvec.vtk import ev2vtk

variables_for_3D = ["X1", "X2", "LA", "pos", "B"]
visu_data_3D = ev[variables_for_3D]
ev2vtk("pyGVEC_3d_visu", visu_data_3D)